# Récupération des données d'un ficher ".csv" pour la création d'in fichier de définition MODBUS pour Webdyn (.ini)

librairies python utilisées...

In [1]:
import csv
import re
from utils import Resize
from modbusDefinition import getVarCoeff, modbusDefinition

**Varoiables globales**
  - "fileCsvName" : nom du fichier à parser  
  - "delimiter" : type de délimiteur (pour le fichier csv à parser)
  - "verbose" : activation des log de debug 

In [2]:
fileCsvName  	= "./csv_inputs/exemple.csv"
delimiter 		= "\t"
verbose = True

## Analyse du ficgier CSV
on ne récupère que les colonnes :
  - **'##'** ou **'#'**, qui permet d'ignorer les lignes où ce champs vaut '#'
  - **'Requête'**, qui correspond au libellé de la requête 
  - **'Variable'**, qui correspond au libellé de la variable
  - **'Type'**, qui contient le type de variable  
  - **'16b'** ou  **'Adr_16b'**, qui contient l'adresse (16 bits)
  - **'N'**, qui contient le nbomre de répétitions d'un champs.  
    (ex : pour mettre une taille max pour les chaines de caractères) 
  - les abscisses et ordonnées de 2 points (**'x1', 'x2', 'y1', et 'y2'** ), ce qui permettent de cacluler les coefs **'varCoeffA'** et **'varCoeffB'**    
    pour le calcul par interpolation linéaire "y = A * x + B" des valeurs à remonter 'y', en connaissant les valeurs lues 'x'.  
  - **'Unité'**, qui contient l'unité des données récupérées  
  

### choix des lignes de variables
Il semble que la colonne "CE"  soit un compteur de variables.   
... on ne prend comme variables que les lignes ayant un CE.  

In [3]:
csvData = {}
with open(fileCsvName) as csvfile:
	reader = csv.DictReader(csvfile, delimiter=delimiter)

	for row in reader:
		#print(row)

		####	récupération des colonnes utiles 	####
		requiredFields = True
		if 'Requête' 	not in row.keys() : requiredFields = False
		if 'Variable' 	not in row.keys() : requiredFields = False
		if 'Type' 		not in row.keys() : requiredFields = False
		if 'Adr_16b' 	not in row.keys() : requiredFields = False

		if requiredFields :
			reqName = str(row['Requête']).replace(" ","_").replace(":","")					
			
			# lignes ingorées 
			if 		'##'	in row.keys() :	ignore_line = row['##']					
			elif 	'#'		in row.keys() :	ignore_line = row['#' ]	
			else :							ignore_line = ""

			# on ne prend comme variables que les lignes qui commencent pas par '#' ou '##'
			if ignore_line not in ["##", "#"] :  		
				varName 		= str(row['Variable']).replace(" ","_").replace(":","")

				if varName		!= "" :  			# ligne avec nom de variable non-vide
					varType	 	= row['Type']		# type 
					varAddr	 	= row['Adr_16b']	# adresse des variables
					varNb	 	= row['N']			# nombre de répétition du champs
					x1			= row['x1']			# abscisse point 1 pour la "mise à l’échelle de la variable (Ax +B)"
					x2			= row['x2']			# abscisse point 2 pour la "mise à l’échelle de la variable (Ax +B)"
					y1			= row['y1']			# ordonnée point 1 pour la "mise à l’échelle de la variable (Ax +B)"
					y2			= row['y2']			# ordonnée point 2 pour la "mise à l’échelle de la variable (Ax +B)"
					unit 		= row['Unité']		# unité
					
					if reqName not in csvData :
						csvData[ reqName ] = {}
					if varName not in csvData[ reqName ] :
						csvData[ reqName ][ varName ] = {}
						
					csvData[ reqName ][ varName ] = { 	
						"type"		: varType	,	# type
						"addr"		: varAddr	,	# adresse des variables
						"N"			: varNb	 	,	# nombre de répétition du champs
						"x1"  		: x1		,	# abscisse point 1 pour la "mise à l’échelle de la variable (Ax +B)"
						"y1"  		: y1		,	# abscisse point 2 pour la "mise à l’échelle de la variable (Ax +B)"
						"x2"  		: x2		,	# ordonnée point 1 pour la "mise à l’échelle de la variable (Ax +B)"
						"y2"  		: y2		,	# ordonnée point 2 pour la "mise à l’échelle de la variable (Ax +B)"
						"unit"		: unit		,	# unité
					}
				#else:	print("Nom de variable vide : " + str(row) )
			else: 		print("ligne ingorée :        " + str(row) )

		else:
			print("Pb dans les colonne du csv : \n" + str(row.keys()) )


ligne ingorée :        {'##': '#', 'Requête': 'ETAT GROUPE', 'Variable': 'Groupe en Auto groupe 1', 'Type': 'bool', 'Adr_16b': '1000', 'N': '', 'x1': '', 'x2': '', 'y1': '', 'y2': '', 'Unité': ''}
ligne ingorée :        {'##': '#', 'Requête': 'ETAT GROUPE', 'Variable': 'Groupe en Auto groupe 2', 'Type': 'bool', 'Adr_16b': '1000', 'N': '', 'x1': '', 'x2': '', 'y1': '', 'y2': '', 'Unité': ''}
ligne ingorée :        {'##': '#', 'Requête': 'ETAT GROUPE', 'Variable': 'réserve', 'Type': '', 'Adr_16b': '1003', 'N': '', 'x1': '', 'x2': '', 'y1': '', 'y2': '', 'Unité': ''}


# Affichage des requêtes trouvés et suppression des requêtes vides

In [4]:
emptyReqList = []
for reqName in csvData:	
	print(" ---- '" + str(reqName) + "' ---- " )
	if len(csvData[reqName]) == 0 : 
		print("   " + Resize("", 60)+ " --> requête vide : on ne la prend pas"  )
		emptyReqList.append(reqName)	# liste des requêtes vides
	else :
		for varName in csvData[reqName] :
			print("   " + Resize(varName, 60) + " --> " + str(csvData[reqName][varName]) )

# suppression des requêtes vides
for r in emptyReqList :
	csvData.pop(r)


 ---- 'DEFAUTS' ---- 
   Défaut_mesure_de_niveau_amont_grille                         --> {'type': 'u16', 'addr': '100', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}
   Défaut_mesure_de_niveau_aval_centrale                        --> {'type': 'u16', 'addr': '101', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}
   Défaut_niveau_bas                                            --> {'type': 'u16', 'addr': '102', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}
   Alarme_niveau_haut                                           --> {'type': 'bool', 'addr': '103', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}
   Alarme_niveau_bas                                            --> {'type': 'bool', 'addr': '104', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}
   Défaut_acquisition_température_centrale                      --> {'type': 'u16', 'addr': '105', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}
   Défaut_acquis

# défintion des constantes 

In [5]:
#(req)id		: "?"	# compteur de requêtes 			-> calculé
#(req)Name		= "?"	# nom de requête 				-> pris dans le fichier csv
ReadFctCode 	= 4
WriteFctCode 	= 0
#StartReg 		= min(addrList)							-> calculé depuis les données du fichier csv
#NbReg			= len(addrList)							-> calculé depuis les données du fichier csv
EnableReading	= 1
EnableWritting	= 2
Option1			= 0
Option2			= 0

#varIndex		= 1		# compteur de variables 		-> calculé
#varReqIndex	= 1		# index du groupe (requête)		-> calculé
#varName		= "?"	# nom de variable 				-> pris   dans le fichier csv
varType			= 8		# type de variable 				-> depuis dans le fichier csv (à vérifier)
varSigned		= 1		# signe de la variable 	-> ...	à prendre dans le fichier csv ???
varPosition		= "?"	# adresse relative				-> depuis dans le fichier csv (à vérifier)
varOption1		= ""
varOption2		= ""
varCoeffA		= 1		# gain pour la variable			-> depuis dans le fichier csv (à vérifier)
varCoeffB		= 0		# offset de la variable			-> depuis dans le fichier csv (à vérifier)
varUnit			= ""	# unité  de la variable			-> pris dans le fichier csv
varAction		= 2




## Fonction de conversion des types de données  
Les types "MW", "MD?", "M" fournis par le SCADA en même temps que l'adresse sont convertis en types "MODUS" (colonne 'varType') 
  
**TODO** :  
  - vérifier la conversion ci-dessous, et trouver à quoi correspond "MD",  
  - vérifier que l'adresse n'est pas impactée (multiplication x2 ou x4 de l'adresser suivant les types ?),   
  - vérifier que le signe n'est pas impacté (colonne MODBUS 'varSigned'),   

In [6]:
def dataTypeTranslate(wordType="u16") :

	d = {		# les adresses sont en mots (de 16 bits)
		"bool"		: 1  ,	# { 'size':  1,	'littleEndian':True,	'signed': 2,	'fr' : "bit", 									}, 	
		"u8"		: 2  ,	# { 'size':  1,	'littleEndian':True,	'signed': 2,	'fr' : "octet", 								}, 
		"u16"		: 3  ,	# { 'size':  1,	'littleEndian':True,	'signed': 2,	'fr' : "mot", 									}, 
		"u16inv"	: 4  ,	# { 'size':  1,	'littleEndian':False,	'signed': 2,	'fr' : "mot inversé", 							}, 
		"u32"		: 5  ,	# { 'size':  2,	'littleEndian':True,	'signed': 2,	'fr' : "double mot", 							}, 
		"u32inv"	: 6  ,	# { 'size':  2,	'littleEndian':False,	'signed': 2,	'fr' : "double mot inversé", 					}, 
		"f32"		: 7  ,	# { 'size':  2,	'littleEndian':True,	'signed': 2,	'fr' : "flottant", 								}, 
		"f32inv"	: 8  ,	# { 'size':  2,	'littleEndian':False,	'signed': 2,	'fr' : "flottant inversé", 						}, 
		"char"		: 9  ,	# { 'size':  1,	'littleEndian':True,	'signed': 2,	'fr' : "chaîne de caractères", 					}, 
	#	"???"		: 10 ,	# { 'size':  1,	'littleEndian':Truee,	'signed': 2,	'fr' : "Format spécifique afficheur Siebert", 	}, 
	}
	try :
		varType = d[wordType]
	except :
		varType = str(wordType) + "???"

	return varType

# Construction du fichier modbus  

**Valeurs d'initialisation**  

2 structures permettent de choisir les valeur d'initialisations des colonnes MODBUS :  
  - pour les requêtes  : **mb.RequestsTableColumns**
  - pour les variables : **mb.VariablesTableColumns**
  
... ici, on ne fait que les affichier
_(mais il est possible de modifier ces valeurs d'init via les attributs ci-dessus)_ 

In [7]:
mb = modbusDefinition("La Serve")
mb.printReqColumns(True)
mb.printVarColumns(True)

RequestsTableColumns...
# id;Name;ReadFctCode;WriteFctCode;StartReg;NbReg;EnableReading;EnableWritting;Option1;Option2
   id                   ---> ?
   Name                 ---> ?
   ReadFctCode          ---> 3
   WriteFctCode         ---> 0
   StartReg             ---> 0
   NbReg                ---> 0
   EnableReading        ---> 1
   EnableWritting       ---> 2
   Option1              ---> 0
   Option2              ---> 0
VariablesTableColumns...
# varIndex;varReqIndex;varName;varType;varSigned;varPosition;varOption1;varOption2;varCoeffA;varCoeffB;varUnit;varAction
   varIndex             ---> ?
   varReqIndex          ---> ?
   varName              ---> ?
   varType              ---> 8
   varSigned            ---> 2
   varPosition          ---> 1
   varOption1           ---> 
   varOption2           ---> 
   varCoeffA            ---> 1
   varCoeffB            ---> 0
   varUnit              ---> 
   varAction            ---> 2


# Décodages des valeurs (trouvées dans le CSV), et appel de la fonction "add_Request_And_Variables_with_adresses()"
L'appel à la fonction "add_Request_And_Variables_with_adresses()" permet d'ajouter d'un seul bloc :   
  - la requête, et 
  - les variables qui lui sont associées,

Cela pemet aussi d'automatiser un certain nombre de calculs :
  - [id] l'indetifiant (compteur) des requêtes,  
  - [StartReg] l'adresses de début de requête,  
  - [NbReg] le comptage du nombre d'octets à lire pour la requête,  
    
  - [varIndex] l'indetifiant (compteur) des variables,  
  - [varReqIndex] l'association des variables à leur requête,  
  - [varPosition] le calcul de l'offset de la variable par rapport àson adresse de début,  
  
_Note : la requête peut alors n'être définie que par son nom [Name], le reste des champs étant calculé ou figés par une valeur d'init._


## Calcul de l'interpolation linéaire (coefs A & B)  
Les colonnes du CSV ont été renommées en abscisses et ordonnées de 2 points dont on fait l'interpolation linéaire via la fonction 'getVarCoeff()' :  
  - x1	<= 'Val.API mini'   
  - x2	<= 'Val. API maxi'  
  - y1	<= 'Val. fin mini'  
  - y2	<= 'Val. fin maxi'  
	
... ce qui permet d'avoir les colonnes 'varCoeffA' et 'varCoeffB' de la definition MODBUS.  
  
## L'unité  
Dans la colonne MODBUS 'varUnit', on reprend telle quelle la colonne CSV 'unit'.  


In [8]:
for reqName in csvData:	
	print(" ---- '" + str(reqName) + "' ---- " )
	print(csvData)
	V = {}


	for varName in csvData[reqName] :
		print("   " + Resize(varName, 60) + " --> " + str(csvData[reqName][varName]) )
		
		### 	décodage des variables	 ###
		wordAddr    	= int(csvData[reqName][varName]['addr'])
		wordType    	= csvData[reqName][varName]['type']

		# coefs A, B à partir des valeurs min/max 
		(A, B) = getVarCoeff(	csvData[reqName][varName]['x1'],						# coefs A & B
								csvData[reqName][varName]['y1'],
								csvData[reqName][varName]['x2'],
								csvData[reqName][varName]['y2']	)

		# construction d'un dictionnaire décrivant la requête (avec les noms de colonne 'modbus')
		newVar = {
			#'varIndex' 	: calculé automatiquement
			#'varReqIndex'	: calculé automatiquement
			'varName'		: varName									,
			'varType'		: dataTypeTranslate(wordType)				, 	# TODO : vérifier la conversion lettre -> chiffre ???
			#'varSigned'	: en dure (VariablesTableColumns)
			#'varPosition'	: calculé automatiquement à partir de l'adresse 'wordAddr'
			#'varOption1'	: en dure (VariablesTableColumns)
			#'varOption2'	: en dure (VariablesTableColumns)				
			'varCoeffA'		: A											,	# mise à l’échelle de la variable (Ax +B)
			'varCoeffB'		: B											,	# mise à l’échelle de la variable (Ax +B)
			'varUnit'		: str(csvData[reqName][varName]['unit'])	,
			#'varAction'	: en dure (VariablesTableColumns)			
			'debug'			: csvData[reqName]
		}

		####	détection des adresses en doubles (dans la définiton des variables)	###
		if wordAddr not in V :
			V[wordAddr] = newVar
		else :
			print("\n")
			print("/!\\ Il exite déjà une variabble à l'adresse " + str(wordAddr) + "... /!\\")
			print(V[wordAddr])
			print(newVar)
			raise TypeError("/!\\ Duplication d'adresse de variables /!\\")


	if reqName :	# suppression des ligne où le nom de requête n'est pas défini...

		if 1 :		# debug
			print("req: " +Resize(reqName) + "...")
			for a in V : 
				print("      @" + Resize(a, 4) + ": " +	mb.strValues(V[a]) )		# affichage des valeurs d'un dictionnaire définissant une variable 

		# ajout de la requêtes (R) et des varibles associées indexées par les adresse (V) 
		mb.add_Request_And_Variables_with_adresses(reqName, V, False)

 ---- 'DEFAUTS' ---- 
{'DEFAUTS': {'Défaut_mesure_de_niveau_amont_grille': {'type': 'u16', 'addr': '100', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}, 'Défaut_mesure_de_niveau_aval_centrale': {'type': 'u16', 'addr': '101', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}, 'Défaut_niveau_bas': {'type': 'u16', 'addr': '102', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}, 'Alarme_niveau_haut': {'type': 'bool', 'addr': '103', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}, 'Alarme_niveau_bas': {'type': 'bool', 'addr': '104', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}, 'Défaut_acquisition_température_centrale': {'type': 'u16', 'addr': '105', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}, 'Défaut_acquisition_température_extérieur': {'type': 'u16', 'addr': '106', 'N': '', 'x1': '', 'y1': '', 'x2': '', 'y2': '', 'unit': ''}, 'Protection_de_découplage': {'type': 'u8', 'addr': '107', 'N': '', 'x1': '', 'y1

**décodage de l'adresse**
  -  utilsation d'un regex '%(\w)(\d+)':
	- '%' ets le début du mot   
	- ([A-Z]+) est un chaine de lettres qui correspond au type de données    
	- ([0-9]+) est un chaine de chiffres qui correspond à l'adresse,  
	 ... on pourrait utiliser (\d+) ?  


# Fichier Modbus

In [9]:
print("------------------------------------------------ Requêtes  ------------------------------------------------")
mb.printRequest('all')

------------------------------------------------ Requêtes  ------------------------------------------------
req1     --- DEFAUTS                             ---> '1;DEFAUTS;3;0;100;8;1;2;0;0'
req2     --- ETAT_GROUPE                         ---> '2;ETAT_GROUPE;3;0;1000;20;1;2;0;0'
 ----------------------------------------- 
   id         Name  ReadFctCode  WriteFctCode  StartReg  NbReg  EnableReading  EnableWritting  Option1  Option2
0   1      DEFAUTS            3             0       100      8              1               2        0        0
1   2  ETAT_GROUPE            3             0      1000     20              1               2        0        0


In [10]:
print("------------------------------------------------ Variables ------------------------------------------------")
mb.printVariables('all')

------------------------------------------------ Variables ------------------------------------------------
----------------------------------------------------------------------------------
req 1 (DEFAUTS) : variables... 
   0;1;Défaut_mesure_de_niveau_amont_grille;3;2;1;;;1.0;0.0;;2
   1;1;Défaut_mesure_de_niveau_aval_centrale;3;2;2;;;1.0;0.0;;2
   2;1;Défaut_niveau_bas;3;2;3;;;1.0;0.0;;2
   3;1;Alarme_niveau_haut;1;2;4;;;1.0;0.0;;2
   4;1;Alarme_niveau_bas;1;2;5;;;1.0;0.0;;2
   5;1;Défaut_acquisition_température_centrale;3;2;6;;;1.0;0.0;;2
   6;1;Défaut_acquisition_température_extérieur;3;2;7;;;1.0;0.0;;2
   7;1;Protection_de_découplage;2;2;8;;;1.0;0.0;;2
   8;1;Protection_de_découplage_longue;2;2;9;;;1.0;0.0;;2
----------------------------------------------------------------------------------
 --- DEFAUTS --- 
   varIndex  varReqIndex                   varName  varType  varSigned  varPosition varOption1 varOption2  varCoeffA  varCoeffB varUnit  varAction
0         0            1  D

## Création du fichier

In [11]:
mb.modbusDefFile(outputDir= "./out/Nouvergies/", fileName = "modbus_La_Serve.ini", verbose = True)

>>> création du répertoire './out/Nouvergies/'

 --- fichier ./out/Nouvergies/modbus_La_Serve.json créé ---- 
#Based on La Serve collected values

# Description of fields
# id;Name;ReadFctCode;WriteFctCode;StartReg;NbReg;EnableReading;EnableWritting;Option1;Option2
Modbus_RequestsTables={
1;DEFAUTS;3;0;100;8;1;2;0;0
2;ETAT_GROUPE;3;0;1000;20;1;2;0;0
}



# Description of fields
# varIndex;varReqIndex;varName;varType;varSigned;varPosition;varOption1;varOption2;varCoeffA;varCoeffB;varUnit;varAction
#
Modbus_VariablesTables={
0;1;Défaut_mesure_de_niveau_amont_grille;3;2;1;;;1.0;0.0;;2
1;1;Défaut_mesure_de_niveau_aval_centrale;3;2;2;;;1.0;0.0;;2
2;1;Défaut_niveau_bas;3;2;3;;;1.0;0.0;;2
3;1;Alarme_niveau_haut;1;2;4;;;1.0;0.0;;2
4;1;Alarme_niveau_bas;1;2;5;;;1.0;0.0;;2
5;1;Défaut_acquisition_température_centrale;3;2;6;;;1.0;0.0;;2
6;1;Défaut_acquisition_température_extérieur;3;2;7;;;1.0;0.0;;2
7;1;Protection_de_découplage;2;2;8;;;1.0;0.0;;2
8;1;Protection_de_découplage_longue;2;2;9;;;1.0;0.0